In [1]:
!pip install keras-tuner

In [2]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.datasets import boston_housing

In [3]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [4]:
x_train.shape

(404, 13)

# Set Hyperparameters using Keras Tuner and Build Model

In [5]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        if i==0:
            model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32,max_value=512,step=32),activation='sigmoid',input_shape=(13,)))
        else:
            model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32,max_value=512,step=32),activation='sigmoid'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.01),
        loss=keras.losses.MeanSquaredError(),
        metrics=[keras.metrics.MeanSquaredError()])
    return model

In [6]:
tuner = RandomSearch(
    build_model,
    objective='mean_squared_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Housing Boston')

In [7]:
tuner.search(x_train, y_train,epochs=5)

Trial 5 Complete [00h 00m 07s]
mean_squared_error: 84.82774861653645

Best mean_squared_error So Far: 84.82774861653645
Total elapsed time: 00h 00m 38s
INFO:tensorflow:Oracle triggered exit


In [8]:
tuner.results_summary()

Results summary
Results in project/Housing Boston
Showing 10 best trials
Objective(name='mean_squared_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 14
units_0: 288
units_1: 480
units_2: 224
units_3: 64
units_4: 320
units_5: 480
units_6: 416
units_7: 448
units_8: 512
units_9: 32
units_10: 288
units_11: 448
units_12: 192
units_13: 192
units_14: 416
units_15: 320
units_16: 448
units_17: 448
units_18: 64
Score: 84.82774861653645
Trial summary
Hyperparameters:
num_layers: 9
units_0: 352
units_1: 448
units_2: 128
units_3: 288
units_4: 320
units_5: 32
units_6: 320
units_7: 512
units_8: 480
units_9: 96
units_10: 448
units_11: 352
units_12: 416
units_13: 480
units_14: 384
units_15: 64
Score: 84.94204966227214
Trial summary
Hyperparameters:
num_layers: 9
units_0: 64
units_1: 320
units_2: 480
units_3: 128
units_4: 224
units_5: 448
units_6: 288
units_7: 160
units_8: 192
units_9: 96
units_10: 320
units_11: 160
units_12: 480
units_13: 320
units_14: 160
units_15: 160
units_16: 2

# Selecting the best model from Tuner

In [9]:
model=tuner.get_best_models(num_models=1)[0]

# Model Summary

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 288)               4032      
                                                                 
 dense_1 (Dense)             (None, 480)               138720    
                                                                 
 dense_2 (Dense)             (None, 224)               107744    
                                                                 
 dense_3 (Dense)             (None, 64)                14400     
                                                                 
 dense_4 (Dense)             (None, 320)               20800     
                                                                 
 dense_5 (Dense)             (None, 480)               154080    
                                                                 
 dense_6 (Dense)             (None, 416)               2

# Evaluating the Model

In [11]:
model.evaluate(x_test,y_test)

4/4 [==============================] - 0s 6ms/step - loss: 85.2188 - mean_squared_error: 85.2188


[85.21875, 85.21875]